writing to dictionary

In [ ]:
s2 = {}
s2.update({"key" : "value"})
s2['k'] = 2
s2['cc'] = {5:8}
s2['cc'].update({'p':8})
s2['10.4.5.6'] = {'h':5}
s2['10.4.5.6'].update({'p':8})

print(s2)

reading flux db to get host kpi details

In [ ]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=2)

def connectToDB():
  #token = 'UnZq8-3qAHW4bk5BNjZgPJLBeeNkOXWatintbu4RAZe_96fdRbPHofP_sE6JWNEPrTnGyFUg26ofUifZQx19DA=='
  mac_token = "tKQMaN6mMBXH-gDotw6qvpEOvcZNIMILQWTH1LTKFDddf3e4owp48cG88bFae1L_H3H5Tp8GV0jrDdzBjQiRhQ=="
  org = "LSBU"
  url = "http://localhost:8086"

  client = influxdb_client.InfluxDBClient(url=url, token=mac_token, org=org)
  return client

def readFromDB(client, bucket):
  
  res = {}

  # To create a empty set you have to use the built in method:
  victim_set = {}
  host_set={}

  # for each host query the DB to get the information
  #querying data from db
  query_api = client.query_api()  
  query = """
  from(bucket: "telemetryData")
	|> range(start: -120m)
  |> filter(fn: (r) => r["_measurement"] == "test7")
  |> filter(fn: (r) => r["_field"] == "cpu_utilization" or r["_field"] == "storage_free" or r["_field"] == "vm_free")
  |> filter(fn: (r) => r["host"] == "10.35.109.155")
  """


  tables = query_api.query(query, org="LSBU")
  records = {}

  for table in tables:
      for record in table.records:
        res[record['_field']] = record['_value']
  records["10.35.109.155"] = res
  print(records)    

reading from dictionary of list and lisi of dictionary

In [3]:
#check if key exists:
docker4 = [{'ctr':[{"major":254,"minor":0,"op":"read","value":4096}, {"major":223,"minor":0,"op":"read","value":4096}], 'Y':[4, 5, 6]}]
print('docker4')

for doc in docker4:
  if 'ctr' in doc:
    for k in doc['ctr']:
        print(k['major'])

docker4
254
223


getting data from stat api of docker

In [4]:
import docker
client = docker.from_env()
stats = client.containers.get('container').stats(stream=False)
UsageDelta = stats['cpu_stats']['cpu_usage']['total_usage'] - stats['precpu_stats']['cpu_usage']['total_usage']
# from informations : UsageDelta = 25382985593 - 25382168431

SystemDelta = stats['cpu_stats']['cpu_usage']['system_cpu_usage'] - stats['precpu_stats']['cpu_usage']['system_cpu_usage']
# from informations : SystemDelta = 75406420000000 - 75400410000000

len_cpu = len(stats['cpu_stats']['cpu_usage']['percpu_usage'])
# from my informations : len_cpu = 2


percentage = (UsageDelta / SystemDelta) * len_cpu * 100
# this is a little big because the result is : 0.02719341098169717

percent = round(percentage, 2)
# now The output is 0.02 and thats the answer.

NotFound: 404 Client Error for http+docker://localhost/v1.41/containers/container/json: Not Found ("No such container: container")

for each container exec python or exec only stress

In [ ]:
import docker

from pprint import PrettyPrinter

pp = PrettyPrinter(indent=2)

client = docker.from_env()

#container = client.containers.get(CONTAINER_ID)
for container in client.containers.list():
    pp.pprint(container.short_id)
    exit_code, output = container.exec_run("ls")
    pp.pprint(output)


python file for while, epoch, random and stress

In [ ]:
from random import randint
import os

def startservice():
        for epoch in range(10):
            cpu = randint(1, 5)
            io = randint(1, 3)
            mem = randint(1, 10)
            vm = randint(100, 300)
            print(cpu,io, mem, vm)

            cmd = f"""
                stress -c {cpu} -i {io} -m {mem} --vm-bytes {vm}M -t 10s
                """
            os.system(cmd)
            #stress -c 2 -i 1 -m 1 --vm-bytes 128M -t 10s

if __name__ == '__main__':
    startservice()

In [ ]:
from random import randint

def startservice():
    while True:
        for epoch in range(10):
            cpu = randint(0, 5)
            io = randint(0, 3)
            mem = randint(0, 10)
            vm = randint(100, 300)
            print(cpu,io, mem, vm)
            #stress -c 2 -i 1 -m 1 --vm-bytes 128M -t 10s

if __name__ == '__main__':
    startservice()

running python file from docker

nano test.py 
update file: example - print("python file")
or docker cp 
apt-get install python3
python3 test.py

from outside container: 
sudo docker exec <id> python3 test.py : worked


creating custom docker image: to add and run script through docker for stress

sudo docker run -it <image-id>
sudo docker cp file <container-id>:<path>/<file-name>
chmod +x file
install stress ng: apt update; apt-get install stress
test stress script
commit: sudo docker commit <id> name:version
save: sudo docker save -o /home/user/test_ubuntu.img name:version

sudo docker load -i img-name
sudo docker run -d name:version

sudo docker exec <container-id> cmd

docker stat 
sudo docker stats <container-id>
?? - screenshot
though API - python

client = docker.DockerClient(base_url='unix:///var/run/docker.sock')

In [ ]:
python3 -m pip install --upgrade pip
pip install docker 
start the docker service first
run one or more dockers : docker run -it 730eeb702b69<image_name>

import docker

client = docker.from_env()
#client.containers.run('alpine', 'echo hello world')
for containers in client.containers.list():
    pp.pprint(containers.stats(decode=None, stream = False))

adding randomness with epoch: while and for-epoch with random

#!/bin/bash
for epoch in {1..5}
do
   echo "Welcome $i times"
done

#!/bin/sh
for i in 1 2 3 4 5
do
  echo "Looping ... number $i"
done

#!/bin/sh
INPUT_STRING=hello
while [ "$INPUT_STRING" != "bye" ]
do
  echo "Please type something in (bye to quit)"
  read INPUT_STRING
  echo "You typed: $INPUT_STRING"
done

#!/bin/sh
while :
do
  echo "Please type something in (^C to quit)"
  read INPUT_STRING
  echo "You typed: $INPUT_STRING"
done

$(( ( RANDOM % 10 )  + 1 ))

$(( ( RANDOM % 10 )  + 1 ))

R=$(($RANDOM%Y+1))


Adding stress component

stress ng
Designed to stress a computer system
Over 220 stress tests 
Designed to be portable

stress ng commands::
apt-get install stress
stress-ng <comp_option> <number> <time_option> <time> : multi options can be combined
# uptime
# stress -c 2 -i 1 -m 1 --vm-bytes 128M -t 10s
# uptime
stress-ng --cpu 2 --vm 4 -t 5m : 

script for stressload: create and execute

touch stressload.sh
nano stressload.sh
# uptime
# stress -c 2 -i 1 -m 1 --vm-bytes 128M -t 10s
# uptime
chmod +x stressload.sh
./stressload.sh


script for dockers:::
scp docker image: ?
------
ry the Python scp module for Paramiko. It's very easy to use. See 
the following example:

    import paramiko
    from scp import SCPClient

    def createSSHClient(server, port, user, password):
        client = paramiko.SSHClient()
        client.load_system_host_keys()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(server, port, user, password)
        return client

    ssh = createSSHClient(server, port, user, password)
    scp = SCPClient(ssh.get_transport())
    
    
Then call scp.get() or scp.put() to do SCP operations.

------
check docker is installed if not install and add user
start dockers from list

sudo apt install -y docker.io
sudo usermod -aG docker $USER

sudo docker images
sudo docker run -d <image-id>

psutil app thread async 